In [1]:
import os
import pickle
from read_mappings import extract_mappings_from_sam, extract_mappings_from_bed, comp_samp_map, comp_bed_bed

Total piscem mappings 1558 (1225)
Total chromap mappings 1120
Total SAM mappings 1165

In [33]:
map_piscem = extract_mappings_from_bed(os.path.join(path, "piscem_analysis/samp1285_k23_psc_off=true_ps_skip=false_thr=0.7/map.bed"))
map_chromap = extract_mappings_from_bed(os.path.join(path, "chromap_analysis/map_out_samp1285.bed"))
map_bowtie = extract_mappings_from_sam(os.path.join(path, "bowtie2/samp1285.sam"))

#### Comparing Piscem with Bowtie

In [15]:
barcode_map = pickle.load(open(os.path.join(path, "pickle_hits/samp1285.pi"), 'rb'))[0]

In [103]:
path="/fs/cbcb-lab/rob/students/noor/Atacseq"
bowtie_piscem=comp_samp_map(os.path.join(path, "bowtie2/samp1285_global.sam"), os.path.join(path, "piscem_analysis/samp1285_k23_psc_off=true_ps_skip=false_thr=0.7/map.bed"),
                            os.path.join(path, "pickle_hits/samp1285.pi"), 10)

count matches are 1097


Lengths of barcodes for which there is a mismatch, missed in piscem, missed in bowtie, missed in both, common and exact match

In [104]:
print(len(bowtie_piscem[0]), len(bowtie_piscem[1]), len(bowtie_piscem[2]), len(bowtie_piscem[3]), len(bowtie_piscem[4]))

16 52 111 9 1097


Matching also includes deduplicates 

Mismatches in piscem total 17

In [14]:
bowtie_piscem[0] ## bowtie, piscem

{'AACTGTGACGCTCAGC': [28429, 28098],
 'AAGACGGTGGCCCTAG': [3518156, 3508736],
 'AATAGGAACGGAATTA': [71729973, 71729967],
 'ACCTCCAACGTTTACG': [2965172, 2970901],
 'ACGCCCAGAAGCCCTG': [180997, 181230],
 'ACGTGGCTGATCAGCT': [3536640, 3527222],
 'ATCGCCGGAGGCCTGT': [20974174, 20973597],
 'ATGGGAGTGTACAGAA': [-1, -1],
 'ATTGCTACTGGAATAT': [-1, -1],
 'CAAGAATGACTGTACA': [16172461, 16171822],
 'CACTTAGGAAAGTCTC': [4667831, 4659866],
 'GGCTTGCGACAACGCC': [-1, -1],
 'GGTGTAACTCGGCTCA': [102671728, 102572647],
 'TCAAAGCACTTCTGAG': [-1, -1],
 'TCAATCGGATACTTTC': [56746112, 56760177],
 'TGGAGTTACGCTTCGG': [-1, -1],
 'TGGGCCATGGAATTGA': [43278803, 43284726]}

#### Comparing Bowtie with Chromap

In [106]:
bowtie_chromap=comp_samp_map(os.path.join(path, "bowtie2/samp1285_global.sam"), os.path.join(path, "chromap_analysis/map_out_samp1285.bed"),
                            os.path.join(path, "pickle_hits/samp1285.pi"),10)
print(len(bowtie_chromap[0]), len(bowtie_chromap[1]), len(bowtie_chromap[2]), len(bowtie_chromap[3]), len(bowtie_chromap[4]))

count matches are 1032
1 132 86 34 1032


Mismatches, missing in chromap, missing in bowtie, missing both, common and exact match

In [107]:
bowtie_chromap[0]

{'GGTGTAACTCGGCTCA': [102671728, 102572651]}

#### Comparing Piscem with Chromap

In [108]:
piscem_chromap= comp_bed_bed(os.path.join(path, "piscem_analysis/samp1285_k23_psc_off=true_ps_skip=false_thr=0.7/map.bed"), os.path.join(path, "chromap_analysis/map_out_samp1285.bed"),
                            os.path.join(path, "pickle_hits/samp1285.pi"), 10)
print(len(piscem_chromap[0]), len(piscem_chromap[1]), len(piscem_chromap[2]), len(piscem_chromap[3]), len(piscem_chromap[4]))

count matches are 1055
39 25 130 36 1055


Mismatches, missing in piscem, missing in chromap, missing both, common

We next check for the above mismatches in bowtie

#### Mismatches

In [23]:
def find_header(barcode_map, barcode) :
    for key, val in barcode_map.items():
        if val == barcode:
            return key
    return None

In [77]:
def match_exists(barcode, map_bed1, map_file2=None, barcode_map=None, mismatch=10):
    map1 = map_bed1[barcode]
    map1 = sorted(map1, key=lambda x:int(x[1]))
    if barcode_map is None:
        if not barcode in map_file2.keys():
            return 0
        map2 = map_file2[barcode]
        map2 = sorted(map2, key=lambda x:int(x[1]))
        i=0
        j=0
        while i < len(map1) and j < len(map2):
            pos1 = int(map1[i][1])
            pos2 = int(map2[j][1])
            if abs(pos1-pos2) <= mismatch:
                return 1
            if pos1 < pos2:
                i+=1
            else:
                j+=1
        return -1
    else:
        header = find_header(barcode_map, barcode)
        if not header in map_file2.keys():                
            return 0
        pos1 = int(map_file2[header]['read1'][1])
        pos2 = int(map_file2[header]['read2'][1])
        i=0
        
        while i < len(map1):
            pos_bed = int(map1[i][1])
            # print(pos1, pos2, pos_bed)
            pos_sam = pos1 if abs(pos1-pos_bed) < abs(pos2-pos_bed) else pos2
            if abs(pos_sam - pos_bed) <= mismatch:
                return 1
            if (pos_bed-pos_sam) > 100:
                break
            i += 1
        return -1    


In [76]:
def count_matches(vv):
    countZ = 0
    countO = 0
    countN = 0
    for l in vv:
        if l==0:
            countZ += 1
        elif l==1:
            countO += 1
        else:
            countN += 1
    return((countZ, countO, countN))
    

For things that don't map in any both piscem and chromap,
only 1 match

In [80]:
print(count_matches([match_exists(x, map_piscem, map_bowtie, barcode_map, 10) for x in piscem_chromap[0].keys()]))
print(count_matches([match_exists(x, map_chromap, map_bowtie, barcode_map, 10) for x in piscem_chromap[0].keys()]))

(38, 1, 0)
(38, 1, 0)


In [95]:
vv=piscem_chromap[0].keys()
header = find_header(barcode_map, list(vv)[-10])
header

'A00836:1008:H5NFJDSX3:1:1240:29387:35321'

In [97]:
print(map_bowtie[header])
print(map_chromap[list(vv)[-10]])
print(map_piscem[list(vv)[-10]])

{'read1': ('chr4', 151099610), 'read2': ('chr4', 151099574)}
[('chr4', 151099577)]
[('chr4', 151099609), ('chr4', 151099609)]


##### Not matched in piscem
All match

In [99]:
print(count_matches([match_exists(x, map_chromap, map_bowtie, barcode_map, 10) for x in piscem_chromap[1]]))

(0, 25, 0)


##### Not matched in chromap
93 match, 12 misalign, 25 not mapped by bowtie

In [101]:
print(count_matches([match_exists(x, map_piscem, map_bowtie, barcode_map, 10) for x in piscem_chromap[2]]))

(25, 93, 12)


### Local Bowtie

In [109]:
map_bowtie = extract_mappings_from_sam(os.path.join(path, "bowtie2/samp1285_local.sam"))

In [113]:
len(map_bowtie)

1220

In [111]:
bowtie_piscem=comp_samp_map(os.path.join(path, "bowtie2/samp1285_local.sam"), os.path.join(path, "piscem_analysis/samp1285_k23_psc_off=true_ps_skip=false_thr=0.7/map.bed"),
                            os.path.join(path, "pickle_hits/samp1285.pi"), 10)
print(len(bowtie_piscem[0]), len(bowtie_piscem[1]), len(bowtie_piscem[2]), len(bowtie_piscem[3]), len(bowtie_piscem[4]))

count matches are 1138
27 55 59 6 1138


In [112]:
bowtie_chromap=comp_samp_map(os.path.join(path, "bowtie2/samp1285_local.sam"), os.path.join(path, "chromap_analysis/map_out_samp1285.bed"),
                            os.path.join(path, "pickle_hits/samp1285.pi"),10)
print(len(bowtie_chromap[0]), len(bowtie_chromap[1]), len(bowtie_chromap[2]), len(bowtie_chromap[3]), len(bowtie_chromap[4]))

count matches are 1075
2 143 42 23 1075


In [114]:
print(count_matches([match_exists(x, map_piscem, map_bowtie, barcode_map, 10) for x in piscem_chromap[0].keys()]))
print(count_matches([match_exists(x, map_chromap, map_bowtie, barcode_map, 10) for x in piscem_chromap[0].keys()]))

(23, 9, 7)
(23, 16, 0)


In [ ]:
print(count_matches([match_exists(x, map_chromap, map_bowtie, barcode_map, 10) for x in piscem_chromap[1]]))

(0, 25, 0)
